In [1]:
from models import FeedforwardNeuralNetModel, TinyCNN, PatternClassifier
import regularizer_losts as rl
from torchvision import datasets, transforms
from torch import optim
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import copy
import utils as CFI_utils
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

import seaborn as sns
colors = sns.color_palette("tab10")

#Logging stuffs
import logging
import sys
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]


#configs
epochs = 3
batch_size = 128
test_batch_size = 10000
use_cuda = True
lr = 1
log_interval = 100
LOAD = True
LOADPATH = 'TinyCNN15:36:27'

#torch specific configs
torch.manual_seed(1)

device = torch.device("cuda")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

## Agenda
### Big question: CFI for NN
### Where are we?
We want to do CFI for NN

## Module 1: The number of activation patterns for a neural networks has to be way smaller than the size of the training/test set.
- Why: If every input results in a different AP, there is no argument for using CFI.
- Progress: with the current regularization effort, we have been able to reduce the number of AP to about 20k for 60k input.
- Aim: Reduce to about few hundreds
- __Tricky part__: Given a network of N layers, we can limit the number of activation patterns by only looking at k < N layers. Question: which k? 

## Module 2: All, or most, of the activation patterns of the test set has to be covered by the AP of the training set
- Why: If the activation patterns in the test set are completely different from the training set, there would be too many false negative at the inferencing time
- Progress: currently not the case. However, we hope that when we can significantly reduce the number of AP in the training set, it would be the case.

## Module 3: Given an AP, how do we shield it?
- Why: Assume we have very few AP, what do we do with it?
- RQ1: Is it possible for inputs of different labels to have the same AP?
- RQ2: If RQ1 is no, then is it possible to have adv. attack on the AP?
- RQ3: 

### Backlog RQs: 
- BRQ1: Can we look at a random k entries in the weight instead of the first k?
- BRQ2: What if we set all the small abs weight in the Pattern classifer to 0? what is the accuracy in that case?
- BRQ3: Given an image and an l2-ball surrounding it, how many activation maps are there? (closely related to RQ1)
- BRQ4: Given a simple attacking method (e.g, Fast gradient sign), build a dataset of activation maps of true and fake digits. Try to train a pattern classifier using that dataset.

In [2]:
#init stuffs
dataset1 = datasets.MNIST('../data', train=True, download=True,
                          transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                          transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

# model = FeedforwardNeuralNetModel(28*28, 128, 10).to(device)
model = TinyCNN().to(device)
